In [1]:
"""
分析所有卷积层 (Conv1, Conv2, Conv3) 的LTR-RT特征
比较不同分辨率下的特征表示
修复版 - 处理所有可能的tensor转换问题
"""

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

def tensor_to_numpy(tensor):
    """
    使用tolist()方法安全转换tensor到numpy
    这是最可靠的方法
    """
    # 如果已经是numpy数组，直接返回
    if isinstance(tensor, np.ndarray):
        return tensor
    
    # 如果是Python列表，转为numpy
    if isinstance(tensor, (list, tuple)):
        return np.array(tensor)
    
    # 处理PyTorch tensor - 使用tolist()方法
    if isinstance(tensor, torch.Tensor):
        # 先detach和cpu（如果需要）
        if tensor.requires_grad:
            tensor = tensor.detach()
        if tensor.is_cuda:
            tensor = tensor.cpu()
        
        # 使用tolist()转为Python列表，再转numpy
        return np.array(tensor.tolist())
    
    # 其他情况
    return np.array(tensor)

# ==================== 配置 ====================
LTR_START_ORIG = 20497
LTR_END_ORIG = 29503

# 各层的映射参数
LAYER_CONFIG = {
    'Conv1': {
        'index': 1,
        'stride': 10,
        'channels': 32,
        'name': 'Conv1 (after MaxPool 10x)'
    },
    'Conv2': {
        'index': 4,
        'stride': 150,  # 10 * 15
        'channels': 64,
        'name': 'Conv2 (after MaxPool 150x)'
    },
    'Conv3': {
        'index': 7,
        'stride': 2250,  # 10 * 15 * 15
        'channels': 128,
        'name': 'Conv3 (after MaxPool 2250x)'
    }
}

# ==================== 加载数据 ====================
print("加载数据...")
mid_res = torch.load('tensor_data.pt')

# 检查数据结构
print(f"\nmid_res类型: {type(mid_res)}")
print(f"mid_res长度: {len(mid_res)}")
print("\n前几个元素的信息:")
for i in range(min(10, len(mid_res))):
    item = mid_res[i]
    if isinstance(item, torch.Tensor):
        print(f"  [{i}] Tensor: shape={item.shape}, device={item.device}")
    elif isinstance(item, (list, tuple)):
        print(f"  [{i}] List/Tuple: 长度={len(item)}")
        if len(item) > 0 and isinstance(item[0], torch.Tensor):
            print(f"       第一个元素: shape={item[0].shape}, device={item[0].device}")
    else:
        print(f"  [{i}] 其他类型: {type(item)}")

# ==================== 分析函数 ====================
def analyze_layer(layer_name, layer_config, mid_res):
    """分析单个卷积层"""
    print(f"\n{'='*60}")
    print(f"分析 {layer_name}")
    print(f"{'='*60}")
    
    try:
        # 提取数据 - 处理可能的嵌套结构
        layer_output = mid_res[layer_config['index']]
        
        print(f"原始数据类型: {type(layer_output)}")
        
        # 如果是列表或元组，取第一个元素
        if isinstance(layer_output, (list, tuple)):
            print(f"  是序列类型，长度={len(layer_output)}")
            tensor = layer_output[0]
        else:
            tensor = layer_output
        
        print(f"Tensor类型: {type(tensor)}")
        print(f"Tensor形状: {tensor.shape}")
        print(f"Tensor设备: {tensor.device}")
        print(f"需要梯度: {tensor.requires_grad}")
        
        # 转换为numpy
        print("正在转换为numpy...")
        data = tensor_to_numpy(tensor)
        print(f"✅ 转换成功！numpy形状: {data.shape}")
        
        # 关键：去除多余的维度 (1, 32, 1, 49981) -> (32, 49981)
        print(f"原始形状: {data.shape}")
        while data.ndim > 2:
            # 找到大小为1的维度并squeeze
            if 1 in data.shape:
                data = np.squeeze(data)
                print(f"  squeeze后: {data.shape}")
            else:
                # 如果没有大小为1的维度，可能需要reshape
                # 假设格式是 (batch, channels, height, width)
                # 取第一个batch
                if data.shape[0] == 1:
                    data = data[0]
                    print(f"  取第一个batch: {data.shape}")
                else:
                    break
        
        # 确保是2D (channels, positions)
        if data.ndim == 3:
            # 如果还是3D，可能是 (channels, 1, positions)
            data = data.squeeze()
            print(f"  最终squeeze: {data.shape}")
        
        print(f"✅ 最终形状: {data.shape}")
        
        stride = layer_config['stride']
        
        # 计算LTR-RT映射
        ltr_start = LTR_START_ORIG // stride
        ltr_end = LTR_END_ORIG // stride
        ltr_width = ltr_end - ltr_start
        
        print(f"\n特征图形状: {data.shape}")
        print(f"LTR-RT原始位置: {LTR_START_ORIG} - {LTR_END_ORIG} bp ({LTR_END_ORIG - LTR_START_ORIG} bp)")
        print(f"LTR-RT特征位置: {ltr_start} - {ltr_end} ({ltr_width} 个位置)")
        print(f"分辨率: 每个特征位置 ≈ {stride} bp")
        
        # 归一化
        print("正在归一化...")
        scaler = preprocessing.RobustScaler()
        data_norm = scaler.fit_transform(data.T).T
        print("✅ 归一化完成")
        
        # 计算选择性
        print("计算选择性指数...")
        ltr_region = data[:, ltr_start:ltr_end]
        bg_indices = list(range(0, ltr_start)) + list(range(ltr_end, data.shape[1]))
        bg_region = data[:, bg_indices]
        
        ltr_activation = ltr_region.mean(axis=1)
        bg_activation = bg_region.mean(axis=1)
        selectivity = (ltr_activation - bg_activation) / (ltr_activation + bg_activation + 1e-8)
        
        # 统计
        ltr_selective = np.sum(selectivity > 0.2)
        bg_selective = np.sum(selectivity < -0.2)
        neutral = np.sum(np.abs(selectivity) <= 0.2)
        
        print(f"\n选择性统计:")
        print(f"  LTR特异性通道 (SI > 0.2): {ltr_selective} ({100*ltr_selective/len(selectivity):.1f}%)")
        print(f"  背景特异性通道 (SI < -0.2): {bg_selective} ({100*bg_selective/len(selectivity):.1f}%)")
        print(f"  中性通道: {neutral} ({100*neutral/len(selectivity):.1f}%)")
        
        top_ch = np.argsort(np.abs(selectivity))[-5:][::-1]
        print(f"\nTop 5 判别通道:")
        for i, ch in enumerate(top_ch, 1):
            print(f"  {i}. Channel {ch}: SI = {selectivity[ch]:+.3f}")
        
        return {
            'data': data,
            'data_norm': data_norm,
            'ltr_start': ltr_start,
            'ltr_end': ltr_end,
            'selectivity': selectivity,
            'top_channels': top_ch,
            'stats': {
                'ltr_selective': ltr_selective,
                'bg_selective': bg_selective,
                'neutral': neutral
            }
        }
    
    except Exception as e:
        print(f"\n❌ 错误: {e}")
        print(f"错误类型: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        return None

# ==================== 分析所有层 ====================
results = {}
for layer_name, config in LAYER_CONFIG.items():
    result = analyze_layer(layer_name, config, mid_res)
    if result is not None:
        results[layer_name] = result
    else:
        print(f"\n⚠️ {layer_name} 分析失败，跳过")

# 检查是否有成功的结果
if len(results) == 0:
    print("\n❌ 所有层都分析失败！")
    print("请检查数据格式和索引是否正确")
    exit(1)

print(f"\n✅ 成功分析了 {len(results)} 个层")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/proj/nobackup/hpc2ns

加载数据...

mid_res类型: <class 'list'>
mid_res长度: 17

前几个元素的信息:
  [0] Tensor: shape=torch.Size([1, 1, 5, 50000]), device=cpu
  [1] Tensor: shape=torch.Size([1, 32, 1, 49981]), device=cpu
  [2] Tensor: shape=torch.Size([1, 32, 1, 49981]), device=cpu
  [3] Tensor: shape=torch.Size([1, 32, 1, 4998]), device=cpu
  [4] Tensor: shape=torch.Size([1, 64, 1, 4979]), device=cpu
  [5] Tensor: shape=torch.Size([1, 64, 1, 4979]), device=cpu
  [6] Tensor: shape=torch.Size([1, 64, 1, 331]), device=cpu
  [7] Tensor: shape=torch.Size([1, 128, 1, 297]), device=cpu
  [8] Tensor: shape=torch.Size([1, 128, 1, 297]), device=cpu
  [9] Tensor: shape=torch.Size([1, 128, 1, 19]), device=cpu

分析 Conv1
原始数据类型: <class 'torch.Tensor'>
Tensor类型: <class 'torch.Tensor'>
Tensor形状: torch.Size([1, 32, 1, 49981])
Tensor设备: cpu
需要梯度: False
正在转换为numpy...
✅ 转换成功！numpy形状: (1, 32, 1, 49981)
原始形状: (1, 32, 1, 49981)
  squeeze后: (32, 49981)
✅ 最终形状: (32, 49981)

特征图形状: (32, 49981)
LTR-RT原始位置: 20497 - 29503 bp (9006 bp)
LTR-RT特征位置: 204

In [2]:
# ==================== 创建对比可视化 ====================
print(f"\n{'='*60}")
print("生成对比可视化...")
print(f"{'='*60}")

fig = plt.figure(figsize=(20, 12))

# 为每层创建3个子图 (热图 + 选择性 + top通道)
row = 0
for layer_name, result in results.items():
    # 热图
    ax_heatmap = plt.subplot(len(results), 3, row*3 + 1)
    im = ax_heatmap.imshow(result['data_norm'], aspect='auto', cmap='RdYlBu_r',
                           interpolation='nearest', vmin=-2, vmax=2)
    ax_heatmap.axvline(result['ltr_start'], color='lime', linewidth=2, linestyle='--')
    ax_heatmap.axvline(result['ltr_end'], color='lime', linewidth=2, linestyle='--')
    ax_heatmap.axvspan(result['ltr_start'], result['ltr_end'], alpha=0.2, color='yellow')
    ax_heatmap.set_title(f"{LAYER_CONFIG[layer_name]['name']}\n特征图", 
                         fontsize=11, fontweight='bold')
    ax_heatmap.set_xlabel('位置')
    ax_heatmap.set_ylabel('通道')
    plt.colorbar(im, ax=ax_heatmap)
    
    # 选择性指数
    ax_si = plt.subplot(len(results), 3, row*3 + 2)
    selectivity = result['selectivity']
    colors = ['red' if s > 0.2 else 'blue' if s < -0.2 else 'lightgray' for s in selectivity]
    ax_si.bar(range(len(selectivity)), selectivity, color=colors, alpha=0.7, width=1.0)
    ax_si.axhline(0, color='black', linewidth=1)
    ax_si.axhline(0.2, color='red', linestyle='--', linewidth=1)
    ax_si.axhline(-0.2, color='blue', linestyle='--', linewidth=1)
    ax_si.set_title(f"选择性指数\n(LTR:{result['stats']['ltr_selective']}, BG:{result['stats']['bg_selective']})", 
                    fontsize=11, fontweight='bold')
    ax_si.set_xlabel('通道')
    ax_si.set_ylabel('SI')
    ax_si.set_ylim(-1, 1)
    ax_si.grid(True, alpha=0.3)
    
    # Top通道激活
    ax_top = plt.subplot(len(results), 3, row*3 + 3)
    for ch in result['top_channels']:
        ax_top.plot(result['data'][ch, :], label=f'Ch {ch}', linewidth=1.5, alpha=0.7)
    ax_top.axvspan(result['ltr_start'], result['ltr_end'], alpha=0.2, color='yellow')
    ax_top.axvline(result['ltr_start'], color='lime', linewidth=2, linestyle='--')
    ax_top.axvline(result['ltr_end'], color='lime', linewidth=2, linestyle='--')
    ax_top.set_title(f"Top 5 判别通道", fontsize=11, fontweight='bold')
    ax_top.set_xlabel('位置')
    ax_top.set_ylabel('激活值')
    ax_top.legend(fontsize=7, loc='best')
    ax_top.grid(True, alpha=0.3)
    
    row += 1

plt.tight_layout()
plt.savefig('all_layers_comparison.png', dpi=300, bbox_inches='tight')
print(f"✅ 对比图已保存: all_layers_comparison.png")
plt.close()



生成对比可视化...


/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 20301 (\N{CJK UNIFIED IDEOGRAPH-4F4D}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 32622 (\N{CJK UNIFIED IDEOGRAPH-7F6E}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 36890 (\N{CJK UNIFIED IDEOGRAPH-901A}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 36947 (\N{CJK UNIFIED IDEOGRAPH-9053}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 29305 (\N{CJK UNIFIED IDEOGRAPH-7279}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: Glyph 24449 (\N{CJK UNIFIED IDEOGRAPH-5F81}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2289553760.py:54: UserWarning: G

✅ 对比图已保存: all_layers_comparison.png


In [3]:
# ==================== 生成汇总表格 ====================
print(f"\n{'='*60}")
print("汇总统计")
print(f"{'='*60}")
print(f"\n{'层名称':<10} {'分辨率':<12} {'LTR宽度':<10} {'LTR特异':<10} {'背景特异':<10} {'最高SI':<10}")
print(f"{'-'*70}")

for layer_name, result in results.items():
    config = LAYER_CONFIG[layer_name]
    max_si = np.max(np.abs(result['selectivity']))
    ltr_width = result['ltr_end'] - result['ltr_start']
    
    print(f"{layer_name:<10} {config['stride']:>4} bp/pos   {ltr_width:>4} pos    "
          f"{result['stats']['ltr_selective']:>4} ch     "
          f"{result['stats']['bg_selective']:>4} ch     "
          f"{max_si:>5.3f}")

# ==================== 创建统计图 ====================
if len(results) > 1:
    fig2, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图: 各层的LTR特异性通道数
    ax1 = axes[0]
    layer_names = list(results.keys())
    ltr_counts = [results[name]['stats']['ltr_selective'] for name in layer_names]
    bg_counts = [results[name]['stats']['bg_selective'] for name in layer_names]
    
    x = np.arange(len(layer_names))
    width = 0.35
    ax1.bar(x - width/2, ltr_counts, width, label='LTR特异性', color='crimson', alpha=0.8)
    ax1.bar(x + width/2, bg_counts, width, label='背景特异性', color='steelblue', alpha=0.8)
    ax1.set_xlabel('卷积层', fontsize=12)
    ax1.set_ylabel('通道数', fontsize=12)
    ax1.set_title('各层选择性通道分布', fontsize=13, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(layer_names)
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # 右图: LTR-RT特征空间宽度
    ax2 = axes[1]
    ltr_widths = [results[name]['ltr_end'] - results[name]['ltr_start'] for name in layer_names]
    strides = [LAYER_CONFIG[name]['stride'] for name in layer_names]
    
    ax2_twin = ax2.twinx()
    ax2.bar(x, ltr_widths, alpha=0.7, color='orange', label='LTR宽度(位置数)')
    ax2_twin.plot(x, strides, 'ro-', linewidth=2, markersize=8, label='降维倍数')
    
    ax2.set_xlabel('卷积层', fontsize=12)
    ax2.set_ylabel('LTR-RT宽度 (特征位置数)', fontsize=12, color='orange')
    ax2_twin.set_ylabel('降维倍数 (bp/位置)', fontsize=12, color='red')
    ax2.set_title('LTR-RT分辨率变化', fontsize=13, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(layer_names)
    ax2.tick_params(axis='y', labelcolor='orange')
    ax2_twin.tick_params(axis='y', labelcolor='red')
    ax2.grid(True, alpha=0.3, axis='y')
    
    lines1, labels1 = ax2.get_legend_handles_labels()
    lines2, labels2 = ax2_twin.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.tight_layout()
    plt.savefig('layers_statistics.png', dpi=300, bbox_inches='tight')
    print(f"\n✅ 统计图已保存: layers_statistics.png")
    plt.close()



汇总统计

层名称        分辨率          LTR宽度      LTR特异      背景特异       最高SI      
----------------------------------------------------------------------
Conv1        10 bp/pos    901 pos       5 ch        7 ch     2.800
Conv2       150 bp/pos     60 pos      12 ch       36 ch     132.287
Conv3      2250 bp/pos      4 pos      22 ch       16 ch     26.802


/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 21367 (\N{CJK UNIFIED IDEOGRAPH-5377}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 31215 (\N{CJK UNIFIED IDEOGRAPH-79EF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 23618 (\N{CJK UNIFIED IDEOGRAPH-5C42}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 36890 (\N{CJK UNIFIED IDEOGRAPH-901A}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 36947 (\N{CJK UNIFIED IDEOGRAPH-9053}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/scratch/ipykernel_942650/2572983950.py:63: UserWarning: G


✅ 统计图已保存: layers_statistics.png


In [4]:
# ==================== 结论 ====================
print(f"\n{'='*60}")
print("🎯 结论与建议")
print(f"{'='*60}")

# 找出最佳分析层
best_layer = max(results.items(), 
                 key=lambda x: x[1]['stats']['ltr_selective'])
print(f"\n✅ 推荐分析层: {best_layer[0]}")
print(f"   原因: 最多的LTR特异性通道 ({best_layer[1]['stats']['ltr_selective']} 个)")

# 给出建议
if best_layer[0] == 'Conv1':
    print("\n💡 建议:")
    print("   - Conv1有最多的选择性通道，说明浅层特征最明显")
    print("   - 可能模型在深层过度平滑了特征")
    print("   - 建议可视化Conv1的卷积核，看学到了什么局部模式")
    
elif best_layer[0] == 'Conv2':
    print("\n💡 建议:")
    print("   - Conv2平衡了分辨率和抽象程度")
    print("   - 这一层可能捕获了中等尺度的LTR-RT结构特征")
    print("   - 建议重点分析Conv2的特征")
    
else:  # Conv3
    print("\n💡 建议:")
    print("   - Conv3有最多的选择性通道，说明深层特征最判别")
    print("   - 模型成功学习到了高度抽象的LTR-RT表示")
    print("   - 这些深层特征可能对应整体的转座子结构")

# 检查是否有问题
total_selective = sum(r['stats']['ltr_selective'] for r in results.values())
if total_selective < 15:
    print("\n⚠️  警告: 选择性通道总数较少")
    print("   可能的原因:")
    print("   - 模型训练不足")
    print("   - 数据标签噪声")
    print("   - LTR-RT特征不够显著")
    print("   建议: 检查模型训练过程和数据质量")

print(f"\n{'='*60}")
print("✅ 分析完成!")
print(f"{'='*60}\n")


🎯 结论与建议

✅ 推荐分析层: Conv3
   原因: 最多的LTR特异性通道 (22 个)

💡 建议:
   - Conv3有最多的选择性通道，说明深层特征最判别
   - 模型成功学习到了高度抽象的LTR-RT表示
   - 这些深层特征可能对应整体的转座子结构

✅ 分析完成!



In [5]:
# ==================== 生成无标亮的纯净热图 ====================
print(f"\n{'='*60}")
print("生成无标亮的纯净热图...")
print(f"{'='*60}\n")

for layer_name, result in results.items():
    data_width = result['data'].shape[1]
    data_channels = result['data'].shape[0]
    
    # 统一图片尺寸 - 所有图都方正
    fig_width = 12
    fig_height = 10
    
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    
    # 绘制热图
    im = ax.imshow(result['data_norm'], aspect='auto', cmap='RdYlBu_r',
                   interpolation='nearest', vmin=-2, vmax=2)
    
    # 计算层描述
    if layer_name == 'Conv1':
        layer_desc = 'Conv1'
    elif layer_name == 'Conv2':
        layer_desc = 'Conv2'
    else:
        layer_desc = 'Conv3'
    
    # 标题
    ax.set_title(f"{layer_desc}\n"
                 f"Shape: {data_channels} channels × {data_width} positions",
                 fontsize=13, fontweight='bold', pad=15)
    ax.set_xlabel('Feature Map Position', fontsize=11)
    ax.set_ylabel('Channel', fontsize=11)
    
    # 颜色条
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Normalized Activation', rotation=270, labelpad=20, fontsize=10)
    
    # 网格 - 根据通道数调整
    if data_channels <= 64:
        ytick_step = max(1, data_channels // 16)
    else:
        ytick_step = max(1, data_channels // 20)
    ax.set_yticks(np.arange(0, data_channels, ytick_step))
    ax.grid(True, alpha=0.2, axis='y', linestyle=':')
    
    plt.tight_layout()
    
    # 保存
    filename = f'{layer_name}_heatmap_clean.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✅ {layer_name} 纯净热图已保存: {filename}")
    print(f"   - 图片尺寸: {fig_width}×{fig_height} inches")
    print(f"   - 特征图: {data_channels} channels × {data_width} positions\n")
    
    plt.close()

print(f"{'='*60}")
print("✅ 完成！所有纯净热图已生成")
print(f"{'='*60}\n")


生成无标亮的纯净热图...

✅ Conv1 纯净热图已保存: Conv1_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 32 channels × 49981 positions

✅ Conv2 纯净热图已保存: Conv2_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 64 channels × 4979 positions

✅ Conv3 纯净热图已保存: Conv3_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 128 channels × 297 positions

✅ 完成！所有纯净热图已生成

